In [1]:
%load_ext twisted_ipython
%autoawait twisted

In [2]:
from twisted.internet import reactor
import attr
from txdbus import client

from pyxsession.dbus.structure import (
    dbus_signature, DBUS_TYPE, DBUS_TYPE_PARAMS, structure, unstructure
)


@attr.s
class Notification:
    appname = attr.ib(default='', metadata={DBUS_TYPE:'s'})
    replaces = attr.ib(default=0, metadata={DBUS_TYPE: 'u'})
    icon = attr.ib(default='', metadata={DBUS_TYPE: 's'})
    summary = attr.ib(default='', metadata={DBUS_TYPE: 's'})
    message = attr.ib(default='', metadata={DBUS_TYPE: 's'})
    actions = attr.ib(default=[], metadata={DBUS_TYPE: 'as'})
    hints = attr.ib(default={}, metadata={DBUS_TYPE: 'ae', DBUS_TYPE_PARAMS: ('s', 'v')})
    timeout = attr.ib(default=1000, metadata={DBUS_TYPE: 'i'})

In [3]:
dbus_signature(Notification)

'susssasa{sv}i'

In [4]:
unstructure(Notification(appname='jupyter', summary='an important message', message='my dog', timeout=5000, hints={'key': 'bro'}))

['jupyter', 0, '', 'an important message', 'my dog', [], {'key': 'bro'}, 5000]

In [5]:
conn = await client.connect(reactor, 'session')

notifier = await conn.getRemoteObject('org.freedesktop.Notifications', '/org/freedesktop/Notifications')

In [6]:
from txdbus.interface import DBusInterface, Method

iface = DBusInterface(
    'org.freedesktop.Notifications',
    Method('Notify', arguments=dbus_signature(Notification), returns='u')
)

unreflected = await conn.getRemoteObject(
    'org.freedesktop.Notifications',
    '/org/freedesktop/Notifications',
    iface
)

await unreflected.callRemote('Notify', *unstructure(Notification(appname='jupyter', summary='an important message', message='my dog', timeout=5000)))

13

In [7]:
structure(
    ['jupyter', 0, '', 'an important message', 'my dog', [], {'key': 'bro'}, 5000],
    Notification
)

Notification(appname='jupyter', replaces=0, icon='', summary='an important message', message='my dog', actions={'key': 'bro'}, hints=5000, timeout=1000)